In [1]:
!pip install wandb
!pip install tensorflow
!pip install keras
import keras
import numpy as np
from keras.datasets import fashion_mnist
from tqdm.auto import tqdm
import tensorflow as tf
import wandb
import pprint

     |████████████████████████████████| 2.0MB 5.1MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
     |████████████████████████████████| 163kB 19.6MB/s 
     |████████████████████████████████| 133kB 17.6MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=28aff39a74bc7529143d5449c78bba87e15655087f37cb9377f0f28b6937cb63
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=5dd3eea42d827e3387477a6259ed86d7a1d0a788afc6def9f731ca72c1538451
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


# Question 1

In [2]:
def load_fashion_mnist(return_images=False):

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

  train_shuffler = np.random.shuffle(np.arange(50000))
  x_train, y_train = x_train[train_shuffler][0], y_train[train_shuffler][0]

  test_shuffler = np.random.shuffle(np.arange(10000))
  x_test, y_test = x_test[test_shuffler][0], y_test[test_shuffler][0]

  x_train = np.array(x_train/255).astype('float32')
  x_test = np.array(x_test/255).astype('float32')

  x_train, x_val = x_train[:50000], x_train[50000:]
  y_train, y_val = y_train[:50000], y_train[50000:]


  if (return_images==False):
    return {
        'train': {
            'X': x_train.reshape([50000, 784]),
            'Y': y_train.reshape([50000])
        },
        'val': {
            'X': x_val.reshape([10000, 784]),
            'Y': y_val.reshape([10000])
        },
        'test': {
            'X': x_test.reshape([10000, 784]),
            'Y': y_test.reshape([10000])
        }
  }

  else :
    return {
      'train': {
          	'X': x_train,
          	'Y': y_train
      },
      'val': {
            'X': x_val,
            'Y': y_val
      },
      'test': {
            'X': x_test,
            'Y': y_test
      }
    }


data = load_fashion_mnist()

4423680/4422102 [==============================] - 0s 0us/step


# Question 2

In [3]:
class neural_network:

  # constructor function - initializes weights
  def __init__(self, dict_layers, initializer):

    self.weights_list = []
    self.biases_list = []
    self.dict_layers= dict_layers

    self.weights_list, self.biases_list = wandb_initializer(dict_layers, self.weights_list, self.biases_list, initializer)

  # function to compute forward propogation
  def forward_prop(self, W, b, X, Y, activation_func):

    A = []
    H = []
    
    H_pre = X
    
    L = self.dict_layers['num_hidden_layers']

    for i in range(L) :
      A.append(W[i] @ H_pre + b[i])
      H_pre = getattr(activation, activation_func)(A[i])
      H.append(H_pre)
    
    A.append(W[L] @ H_pre + b[L])
    
    Y_hat = activation.softmax(A[L])
    
    return {
        'A' : A,
        'H' : H,
        'Y_hat' : Y_hat
    }

  # helper function to perform forward propogation 
  def self_forward_prop(self, X, Y, activation_func) :

    temp = self.forward_prop(self.weights_list,self.biases_list, X, Y, activation_func)
    return temp

  # function to perform backward propogration
  def back_prop(self, W, b, A, H, Y_hat, X, Y,activation_func):

    batch_size = len(Y)
    
    del_w = []
    del_b = []
    L = self.dict_layers['num_hidden_layers']
    
    E = np.zeros(Y_hat.shape)
    E[Y,np.arange(batch_size)] = 1
    
    grad_A = -(E - Y_hat)

    for i in range(L,-1,-1) :

      temp1 = grad_A.reshape(-1,batch_size)
      
      if i==0 :
        temp2 = X.T
      else :
        temp2 = H[i-1].reshape((batch_size ,-1))

      del_w.append(temp1 @ temp2)
      del_b.append(grad_A)

      if(i!=0) :
        grad_H = W[i].T @ grad_A      
        grad_A = grad_H * getattr(activation,activation_func+'_der')(H[i-1])

    return {
        'dw' : del_w,
        'db' : del_b
    }

  # helper function to perform backward propogation
  def self_back_prop(self, A, H, Y_hat, X, Y,activation_func) :
    temp = self.back_prop(self.weights_list,self.biases_list, A, H, Y_hat, X, Y, activation_func)
    return temp

  #  function to compute gradient
  def grad_wandb(self, W, b, X, Y,activation_func):

    X = X.T.reshape((784,-1))
    
    temp = self.forward_prop(W, b, X, Y, activation_func)
    temp2 = self.back_prop(W, b, temp['A'], temp['H'], temp['Y_hat'], X, Y, activation_func)

    return {
        'dw' : temp2['dw'],
        'db' : temp2['db']
    }

  # helper function to compute gradient
  def self_grad_wandb(self, X, Y, activation_func) :
    temp = self.grad_wandb(self.weights_list, self.biases_list, X, Y,activation_func)
    return temp

  # function to compute predictions
  def predict(self, X, activation_func):
    X = X.T.reshape((784,-1))
    temp = self.forward_prop(self.weights_list,self.biases_list, X, 0, activation_func)
    return {
      'Y' : np.argmax(temp['Y_hat'],axis=0),
      'Y_hat' : temp['Y_hat']
    }

  # function to update weights and biases
  def update_vals(self,dw,db) :
    L = len(self.weights_list)
    for i in range(L) :
      # print('dw['+str(L-i-1)+']',dw[L-i-1])
      self.weights_list[i] =self.weights_list[i] - dw[L-i-1].reshape(self.weights_list[i].shape)

    for i in range(len(self.biases_list)) :
      # print('db['+str(L-i-1)+']',db[L-i-1])
      self.biases_list[i] =self.biases_list[i] - db[L-i-1].reshape(self.biases_list[i].shape)
##################################################################################
class activation:
  
  @staticmethod
  def sigmoid(z):
    return 1 / (1 + np.exp(-z))
  
  @staticmethod
  def relu(z):
    return (z>0) * z

  @staticmethod
  def tanh(z):
    return np.tanh(z)

  @staticmethod
  def sigmoid_der(z) :
    return z * (1-z)
  
  @staticmethod
  def relu_der(z) :
    return (z>0)

  @staticmethod
  def tanh_der(z):
    return 1 - z*z

  @staticmethod
  def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x,axis=0)
    #return np.array(tf.nn.softmax(np.array(x)))
##################################################################################
def set_nn_shape(verbose=True, num_hidden_layers=-1, hidden_layer_size=-1):

  input_layer_size = 784
  hidden_layer_size = hidden_layer_size
  num_hidden_layers = num_hidden_layers
  output_layer_size = 10
  
  # input_layer_size = 3
  # hidden_layer_size = hidden_layer_size
  # num_hidden_layers = num_hidden_layers
  # output_layer_size = 2
  if (verbose):
    print("\nNumber Of Hidden Layers:")
    num_hidden_layers = int(input())

    print("\nSize Of Each Hidden Layer:")
    hidden_layer_size = int(input())

    print(f"\nThe Neural Network Has {num_hidden_layers+2} Layers In Total!")
  
  return {"input_layer_size": input_layer_size, "hidden_layer_size": hidden_layer_size, "output_layer_size": output_layer_size, "num_hidden_layers": num_hidden_layers}


In [4]:
def wandb_initializer(nn_shape, weights_list, biases_list, type='random', mu = 0, sigma = 1):
  
  # random initialization
  if (type=='random'):
    initializer = tf.keras.initializers.TruncatedNormal(mean=mu, stddev=sigma)
  # xavier initialization
  elif (type=='xavier'):
    initializer = tf.keras.initializers.GlorotNormal()

  weights_list.append(initializer(shape=(nn_shape['hidden_layer_size'], nn_shape['input_layer_size'])).numpy())
  biases_list.append(initializer(shape=(nn_shape['hidden_layer_size'], 1)).numpy())
  for i in range(nn_shape['num_hidden_layers'] - 1):
    weights_list.append(initializer(shape=(nn_shape['hidden_layer_size'], nn_shape['hidden_layer_size'])).numpy())
    biases_list.append(initializer(shape=(nn_shape['hidden_layer_size'], 1)).numpy())

  weights_list.append(initializer(shape=(nn_shape['output_layer_size'], nn_shape['hidden_layer_size'])).numpy())
  biases_list.append(initializer(shape=(nn_shape['output_layer_size'], 1)).numpy())

  return weights_list, biases_list

In [5]:
class optimizer:

  @staticmethod
  def sgd_old(network, data, config):
    
    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      dw = []
      db = []
      check = 0

      for k in tqdm(range(len(X_train))) :
        x = X_train[k]
        y = Y_train[k]
        temp = network.self_grad_wandb(x,y,activation_func)

        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check=1

        else :
          for j in range(len(dw)) :
            dw[j] =dw[j] + temp['dw'][j]
            db[j] =db[j] + temp['db'][j]

        if (k+1)%batch_size == 0 :
          for dd in dw :
            dd *=eta
          for dd in db :
            dd *=eta
          network.update_vals(dw, db)
          check=0
  ##################################################################################
  @staticmethod
  def sgd(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      for k in tqdm(range(0, len(X_train), batch_size)) :
        
        X = X_train[k: k+batch_size]
        Y = Y_train[k: k+batch_size]
        
        temp = network.self_grad_wandb(X, Y, activation_func)
        
        for j in range(len(temp['db'])) :
          temp['db'][j] = np.sum(temp['db'][j],axis=1)
          
        dw = temp['dw']
        db = temp['db']
        for dd in dw :
          dd*= eta
        for dd in db :
          dd*=eta

        
        network.update_vals(dw, db)
        
      report = run_callback(network, data, config) 
        
      wandb.log({
            'batch_size': config.batch_size, 
            'val_loss' : report['loss']['val'], 
            'train_loss': report['loss']['train'],
            'train_acc': report['accuracy']['train'],
            'val_acc': report['accuracy']['val']  
      }) 
        
        

  ##################################################################################
  @staticmethod
  def momentum(X,Y,max_epochs,eta,gamma,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    dw = []
    db = []
    pred1 = []
    check = 0
    for i in range(max_epochs) :
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          for dd in db :
            dd*= eta
          for dd in dw :
            dd*=eta
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += eta*temp['dw'][j]
            db[j] += eta*temp['db'][j]
        if (k+1) % batch_size == 0 or k == len(X)-1:
          network.update_vals(dw,db)
          for dd in db :
            dd *=gamma
          for dd in dw :
            dd*=gamma

      pred = 0
      for x,y in zip(X,Y) :
        pred = pred + (network.predict(x)!=y)

      print('error',pred)
    
    return pred1
  ################################################################################## 
  @staticmethod
  def NAG(X,Y,max_epochs,eta,gamma,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    dw = []
    db = []
    check = 0
    for i in range(max_epochs) :
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          for dd in db :
            dd*= eta
          for dd in dw :
            dd*=eta
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += eta*temp['dw'][j]
            db[j] += eta*temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          network.update_vals(dw,db)
          for dd in db :
            dd*=gamma
          for dd in dw :
            dd*=gamma
          network.update_vals(dw,db)

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)
  ##################################################################################
  @staticmethod
  def RMSprop(X,Y,max_epochs,eta,beta,batch_size,epsilon) :
    test = get_nn_shape()
    network = neural_network(test,test)
    v_dw = []
    v_db = []
    check1 = 0
    for i in range(max_epochs) :
      dw = []
      db = []
      check = 0
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += temp['dw'][j]
            db[j] += temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          if check1==0 :
            for j in range(len(dw)) :
              v_dw.append( (1-beta)*(dw[j]**2) )
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db.append( (1-beta)*(db[j]**2) )
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

            check1 = 1

          else :
            for j in range(len(dw)) :
              v_dw[j] *= beta
              v_dw[j] += (1-beta)*(dw[j]**2) 
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db[j] *= beta
              v_db[j] += (1-beta)*(db[j]**2) 
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

          network.update_vals(dw,db)
          check = 1

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)

  ##################################################################################
  @staticmethod
  def adam_old(X,Y,max_epochs,eta,beta1,beta2,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    v_dw = []
    v_db = []
    m_dw = []
    m_db = []
    check1 = 0
    num_updates = 0
    for i in range(max_epochs) :
      dw = []
      db = []
      check = 0
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += temp['dw'][j]
            db[j] += temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          if check1==0 :
            for j in range(len(dw)) :
              v_dw.append( (1-beta1)*(dw[j]**2) )
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db.append( (1-beta1)*(db[j]**2) )
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

            check1 = 1

          else :
            for j in range(len(dw)) :
              v_dw[j] *= beta1
              v_dw[j] += (1-beta1)*(dw[j]**2) 
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db[j] *= beta1
              v_db[j] += (1-beta1)*(db[j]**2) 
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

          network.update_vals(dw,db)
          check = 1

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)
  ##################################################################################  
  @staticmethod
  def adam(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      for k in tqdm(range(0, len(X_train), batch_size)) :
        
        X = X_train[k: k+batch_size]
        Y = Y_train[k: k+batch_size]
        
        temp = network.self_grad_wandb(X, Y, activation_func)
        
        for j in range(len(temp['db'])) :
          temp['db'][j] = np.sum(temp['db'][j],axis=1)
          
        dw = temp['dw']
        db = temp['db']
        for dd in dw :
          dd*= eta
        for dd in db :
          dd*=eta

        
        network.update_vals(dw, db)
        
      report = run_callback(network, data, config) 
        
      wandb.log({
            'batch_size': config.batch_size, 
            'val_loss' : report['loss']['val'], 
            'train_loss': report['loss']['train'],
            'train_acc': report['accuracy']['train'],
            'val_acc': report['accuracy']['val']  
      }) 
        


In [6]:
'''
# X = np.array([[1,1,2],[-1,2,3],[10,-67,43],[-5,45,-67]])
# Y = np.array([1,0,1,0])
# temp = solver.sgd(X,Y,100,1e-3)
X=data['train']['X']
# print(len(X))
# print(X[0])
Y = data['train']['Y']
temp = solver.RMSprop(X,Y,100,0.001,0.9,32,1e-2)
print(temp)
'''

"\n# X = np.array([[1,1,2],[-1,2,3],[10,-67,43],[-5,45,-67]])\n# Y = np.array([1,0,1,0])\n# temp = solver.sgd(X,Y,100,1e-3)\nX=data['train']['X']\n# print(len(X))\n# print(X[0])\nY = data['train']['Y']\ntemp = solver.RMSprop(X,Y,100,0.001,0.9,32,1e-2)\nprint(temp)\n"

In [7]:
def run_callback(network,data,config) :
    
    activation_func = config['activation']
    
    X_train = data['train']['X']
    Y_train = data['train']['Y']

    X_val = data['val']['X']
    Y_val = data['val']['Y']

    train_loss = 0
    train_count = 0
    train_sq_error = 0
    
    temp = network.predict(X_train,activation_func)
    train_count = np.sum(temp['Y'].reshape(Y_train.shape)==Y_train)
    
    Y_pred = np.array(temp['Y_hat'].T)
    train_loss = np.sum(-np.log(Y_pred[np.arange(len(X_train)),Y_train]))
    
    #temp['y_hat'][y] = 1 - temp['y_hat'][y]
    #train_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat']))

    val_loss = 0
    val_count = 0
    val_sq_error = 0
    
    temp = network.predict(X_val, activation_func)
    val_count = np.sum(temp['Y'].reshape(Y_val.shape)==Y_val)
    
    Y_pred = np.array(temp['Y_hat'].T)
    val_loss = np.sum(-np.log(Y_pred[np.arange(len(X_val)),Y_val]))
    
    # for x,y in zip(X_val,Y_val) :
    #     temp = network.predict(x,activation_func)
    #     if temp['y'] == y :
    #         val_count += 1
    #     val_loss -= np.log(temp['y_hat'][y]) 
    #     temp['y_hat'][y] = 1 - temp['y_hat'][y]
    #     val_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    
    '''
    test_loss = 0
    test_count = 0
    test_sq_error = 0
    for x,y in zip(X_test,Y_test) :
        temp = network.predict(x,activation_func)
        if temp['y'] == y :
            test_count += 1
        test_loss -= np.log(temp['y_hat'][y]) 
        temp['y_hat'][y] = 1 - temp['y_hat'][y]
        test_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    '''
    
    return  {
        'loss': {
            'train' : train_loss / len(X_train),
            'val' : val_loss / len(X_val)
        },
        'accuracy': {
            'train': train_count / len(X_train),
            'val': val_count / len(X_val)
        }
        #'test' : np.array([test_sq_error,test_loss,test_count])/len(X_test)
    }

    


In [8]:
'''
nn = set_nn_shape()
network = neural_network(nn, 'random')
#data1 = {'train' : {'X': np.array([[1,1,2],[-1,2,3],[10,-67,43],[-5,45,-67],[5,6,7]]), 'Y' : np.array([1,0,1,0,1]) },'val' : {'X': np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]), 'Y' : np.array([0,1,3,2,1]) }}
# need to change this bit later to accomodate other optimization functions

config1 = {'num_epochs' : 5,'lr' : 1e-2,'optimizer': 'sgd', 'batch_size' : 32 , 'weights_initializer' : 'random' , 'weight_decay' : 0.001, 'activation' : 'sigmoid' }
optimizer.sgd(network, data,config1 )

# generating reports for the run
report = run_callback(network, data, config1) 

print(report)
'''

"\nnn = set_nn_shape()\nnetwork = neural_network(nn, 'random')\n#data1 = {'train' : {'X': np.array([[1,1,2],[-1,2,3],[10,-67,43],[-5,45,-67],[5,6,7]]), 'Y' : np.array([1,0,1,0,1]) },'val' : {'X': np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]), 'Y' : np.array([0,1,3,2,1]) }}\n# need to change this bit later to accomodate other optimization functions\n\nconfig1 = {'num_epochs' : 5,'lr' : 1e-2,'optimizer': 'sgd', 'batch_size' : 32 , 'weights_initializer' : 'random' , 'weight_decay' : 0.001, 'activation' : 'sigmoid' }\noptimizer.sgd(network, data,config1 )\n\n# generating reports for the run\nreport = run_callback(network, data, config1) \n\nprint(report)\n"

In [9]:
'''
# generating reports for the run
report = run_callback(network, data, config1) 
print(report)
'''

'\n# generating reports for the run\nreport = run_callback(network, data, config1) \nprint(report)\n'

In [10]:
activation.softmax(np.array([[0.7,0.5],[3,7]]))

array([[0.09112296, 0.00150118],
       [0.90887704, 0.99849882]])

In [11]:
sweep_config = {
    'method': 'random',

    'parameters': {
        'num_epochs': {
            'values': [5, 10]
        },
        'num_hidden_layers': {
            'values': [3, 4, 5]
        },
        'hidden_layer_size': {
            'values': [32, 64, 128]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.5]
        },
        'lr': {
            'values': [1e-3, 1e-4, 1e-5, 1e-6]
        },
        'optimizer': {
            'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'weights_initializer': {
            'values': ['random', 'xavier']
        },
        'activation': {
            'values': ['sigmoid', 'tanh', 'relu']
        }        
    }
}

In [12]:
sweep_id = wandb.sweep(sweep_config, project='test4')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 352vuuwe
Sweep URL: https://wandb.ai/ramkamal/test4/sweeps/352vuuwe


In [13]:
pprint.pprint(sweep_config)

{'method': 'random',
 'parameters': {'activation': {'values': ['sigmoid', 'tanh', 'relu']},
                'batch_size': {'values': [16, 32, 64]},
                'hidden_layer_size': {'values': [32, 64, 128]},
                'lr': {'values': [0.001, 0.0001, 1e-05, 1e-06]},
                'num_epochs': {'values': [5, 10]},
                'num_hidden_layers': {'values': [3, 4, 5]},
                'optimizer': {'values': ['sgd',
                                         'momentum',
                                         'nesterov',
                                         'rmsprop',
                                         'adam',
                                         'nadam']},
                'weight_decay': {'values': [0, 0.0005, 0.5]},
                'weights_initializer': {'values': ['random', 'xavier']}}}


In [14]:
class sweep_module:
  @staticmethod
  def train(config=None):

    with wandb.init(config):
      # print('Checkpoint 1')
      config = wandb.config
      # print('Checkpoint 2')
      
      nn_shape = set_nn_shape(False, config['num_hidden_layers'] , config['hidden_layer_size'])
      
      # print('Checkpoint 3')
      
      # print(nn_shape)
      network = neural_network(nn_shape, config['weights_initializer'])
      # print('Checkpoint 4')
      
      # need to change this bit later to accomodate other optimization functions
      optimizer.sgd(network, data, config)
      
      print('checkpoint 1')
      # generating reports for the run
    '''
      report = run_callback(network, data, config) 
      print(report)
    
      # keeping track of the best model
      if (report['accuracy']['val'] > val_acc_best):
        val_acc_best, network_best = report['accuracy']['val'], network
        
    
      wandb.log({
        'batch_size': config.batch_size, 
        'val_loss' : report['loss']['val'], 
        'train_loss': report['loss']['train'],
        'train_acc': report['accuracy']['train'],
        'val_acc': report['accuracy']['val']  
      })
      '''

In [15]:
# for logging the best model
network_best = None
val_acc_best = -1

# performing the sweep
wandb.agent(sweep_id, sweep_module.train, count=15)

wandb: Agent Starting Run: p6psgb8o with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.0001
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0
wandb: 	weights_initializer: random
wandb: Currently logged in as: ramkamal (use `wandb login --relogin` to force relogin)


Checkpoint 1
Checkpoint 2
Checkpoint 3
{'input_layer_size': 784, 'hidden_layer_size': 64, 'output_layer_size': 10, 'num_hidden_layers': 3}
Checkpoint 4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide



checkpoint 1


batch_size,32
val_loss,nan
train_loss,nan
train_acc,0.09954
val_acc,0.1023
_runtime,42
_timestamp,1615649977
_step,9


batch_size,▁▁▁▁▁▁▁▁▁▁
val_loss,
train_loss,
train_acc,▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▃▄▄▅▆▆▇█
_timestamp,▁▂▃▄▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 86fiakoy with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.001
wandb: 	num_epochs: 5
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.5
wandb: 	weights_initializer: xavier


Checkpoint 1
Checkpoint 2
Checkpoint 3
{'input_layer_size': 784, 'hidden_layer_size': 64, 'output_layer_size': 10, 'num_hidden_layers': 4}
Checkpoint 4


wandb: Ctrl + C detected. Stopping sweep.


In [16]:
nn_shape = {'input_layer_size': 784, 'hidden_layer_size': 64, 'output_layer_size': 10, 'num_hidden_layers': 5}

In [ ]:
init = 'random'

In [ ]:
neural_network(nn_shape, init)